In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Window
from delta.tables import DeltaTable
from delta.pip_utils import configure_spark_with_delta_pip

warehousePath = "extract/07iceberg"

builder = (SparkSession
        .builder
        .appName("IcebergSession")
        .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.local.type", "hadoop")
        .config("spark.sql.catalog.local.warehouse", warehousePath)
        )

jars = [
    "org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.2.1", # run iceberg
]

spark = configure_spark_with_delta_pip(
            spark_session_builder=builder
            ,extra_packages=jars).getOrCreate()

## Create Iceberg Table

In [ ]:
spark.sql("""
CREATE OR REPLACE TABLE local.db.iceberg (
    id BIGINT, 
    data STRING) 
USING ICEBERG 
PARTITIONED BY (data)
TBLPROPERTIES ('key'='value')
""")

## Insert Data

In [ ]:
spark.sql("""
INSERT INTO local.db.iceberg 
VALUES 
    (1, 'a'), 
    (2, 'b')
""")

## Read Data

In [ ]:
spark.sql("SELECT * FROM local.db.iceberg").toPandas()

In [ ]:
spark.table("local.db.iceberg").toPandas()

In [ ]:
spark.read.format("iceberg").load("local.db.iceberg").toPandas()

## Drop Table

In [ ]:
spark.sql("DROP TABLE local.db.iceberg")

In [ ]:
spark.sql("SELECT * FROM local.db.iceberg").toPandas()